In [7]:
import numpy as np
from Server import Server

In [8]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [9]:
from ga import input_wrapper, solve
from helpers import parse
from pruner import magic

In [10]:
path = 'input/dc.in'

In [11]:
def input_wrapper(inp):
    servers = []
    for i, v in enumerate(inp):
        r, slot, id = v[0]
        s, cap = v[1][0], v[1][1]
        serv = Server(id, s, cap)
        serv.row = r
        serv.slot = slot
        servers.append(serv)
    return servers

In [12]:
R, S, U, P, M, unavaiable, servers = parse(path)
servers = input_wrapper(magic(path))

In [13]:
servers

[S[0]: size=2, capacity=40, row=0, slot=0 | P[-1],
 S[1]: size=2, capacity=40, row=0, slot=2 | P[-1],
 S[2]: size=3, capacity=60, row=0, slot=4 | P[-1],
 S[3]: size=2, capacity=40, row=0, slot=7 | P[-1],
 S[4]: size=4, capacity=80, row=0, slot=11 | P[-1],
 S[5]: size=1, capacity=20, row=0, slot=9 | P[-1],
 S[6]: size=5, capacity=100, row=0, slot=15 | P[-1],
 S[7]: size=3, capacity=60, row=0, slot=20 | P[-1],
 S[8]: size=2, capacity=40, row=0, slot=23 | P[-1],
 S[9]: size=3, capacity=60, row=0, slot=25 | P[-1],
 S[10]: size=5, capacity=100, row=0, slot=28 | P[-1],
 S[11]: size=2, capacity=40, row=0, slot=33 | P[-1],
 S[12]: size=3, capacity=60, row=0, slot=35 | P[-1],
 S[13]: size=1, capacity=20, row=0, slot=38 | P[-1],
 S[14]: size=1, capacity=20, row=0, slot=39 | P[-1],
 S[15]: size=3, capacity=60, row=0, slot=43 | P[-1],
 S[16]: size=3, capacity=60, row=0, slot=48 | P[-1],
 S[17]: size=4, capacity=80, row=0, slot=52 | P[-1],
 S[18]: size=4, capacity=80, row=0, slot=56 | P[-1],
 S[19]

In [110]:
def initialize_population(population_size, M, P):
    return [np.random.randint(0, P, size=M) for _ in range(population_size)]

def score_population(population, capacities, rows, M, P, R):
    return [fitness(candidate, capacities, rows, M, P, R) for candidate in population]

def selection(population, scores, retain_frac=0.8, retain_random=0.05):
    retain_len = int(len(scores) * retain_frac)
    sorted_indices = np.argsort(scores)[::-1]
    population = [population[idx] for idx in sorted_indices]
    selected = population[:retain_len]
    leftovers = population[retain_len:]

    for gene in leftovers:
        if np.random.rand() < retain_random:
            selected.append(gene)
    return selected

def mutate(candidate, M, P, mutate_frac=0.1):
    frac = int(M * mutate_frac)
    if frac == 0:
        return
    n_tries = np.random.randint(0, int(len(candidate) * mutate_frac))

    idxs = np.random.randint(0, len(candidate), size=n_tries)
    pools = np.random.randint(0, P, size=n_tries)
    for i in range(n_tries):
        candidate[idxs[i]] = pools[i]

def crossover(mom, dad):
    select_mask = np.random.binomial(1, 0.5, size=len(mom)).astype('bool')
    child1, child2 = np.copy(mom), np.copy(dad)
    child1[select_mask] = dad[select_mask]
    child2[select_mask] = mom[select_mask]
    return child1, child2

def evolve(population, capacities, rows, M, P, R, retain_frac=0.8, retain_random=0.05, mutate_chance=0.05):
    """
    Evolution step
    :param population: list or candidate solutions
    :param target: 20x20 array that represents field in stopping condition
    :param delta: number of steps to revert
    :param retain_frac: percent of top individuals to proceed into the next genetation
    :param retain_random: chance of retaining sub-optimal individual
    :param mutate_chance: chance of mutating the particular individual
    :return: new generation of the same size
    """
    scores = score_population(population, capacities, rows, M, P, R)
    next_population = selection(population, scores, retain_frac=retain_frac, retain_random=retain_random)
    
    # mutate everyone expecting for the best candidate
    for gene in next_population[1:]:
        if np.random.random() < mutate_chance:
            mutate(gene, M, P)

    places_left = len(population) - len(next_population)
    children = []
    parent_max_idx = len(next_population) - 1
    while len(children) < places_left:
        mom_idx, dad_idx = np.random.randint(0, parent_max_idx, 2)
        if mom_idx != dad_idx:
            child1, child2 = crossover(next_population[mom_idx], next_population[dad_idx])
            children.append(child1)
            if len(children) < places_left:
                children.append(child2)
    next_population.extend(children)
    return next_population

def solve(servers, capacities, rows, M, P, R, population_size=10, n_generations=100):
    population = initialize_population(population_size, M, P)
    for generation in range(n_generations):
        population = evolve(population, capacities, rows, M, P, R)
        if generation == 0:
            print("Generation #: best score")
        elif generation % 200 == 0:
            print("Generation ", generation, ": ", fitness(population[0], capacities, rows, M, P, R))
    return population[0]

In [111]:
%%cython -a
import numpy as np
cimport numpy as np
from Server import Server

cpdef int fitness(np.ndarray[long, ndim=1] pools, np.ndarray[long, ndim=1] capacities, np.ndarray[long, ndim=1] rows, int M, int P, int R):
    cdef:
        int least_gc = 1000
        np.int32_t i, j, gci, cap, r, strike_cap
    for i in range(P):
        gci = 1000
        cap = 0
        for j in range(M):
            if pools[j] == i:
                cap += capacities[j]
        for r in range(R):
            strike_cap = 0
            for j in range(M):
                if pools[j] == i and rows[j] == r:
                    strike_cap += capacities[j]
            if gci > (cap - strike_cap):
                gci = cap - strike_cap
        if gci < least_gc:
            least_gc = gci
    return least_gc

In [112]:
servs = [serv.copy() for serv in servers]
capacities = np.array([int(serv.capacity) for serv in servers])
rows = np.array([serv.row for serv in servers])

In [114]:
result = solve(servs, capacities, rows, len(servers), P, R, n_generations=30000)

Generation #: best score
Generation  200 :  255
Generation  400 :  255
Generation  600 :  255
Generation  800 :  255
Generation  1000 :  255
Generation  1200 :  259
Generation  1400 :  259
Generation  1600 :  266
Generation  1800 :  285
Generation  2000 :  287
Generation  2200 :  287
Generation  2400 :  287
Generation  2600 :  292
Generation  2800 :  292
Generation  3000 :  292
Generation  3200 :  292
Generation  3400 :  293
Generation  3600 :  293
Generation  3800 :  293
Generation  4000 :  296
Generation  4200 :  296
Generation  4400 :  296
Generation  4600 :  296
Generation  4800 :  296
Generation  5000 :  296
Generation  5200 :  296
Generation  5400 :  296
Generation  5600 :  296
Generation  5800 :  296
Generation  6000 :  296
Generation  6200 :  296
Generation  6400 :  296
Generation  6600 :  296
Generation  6800 :  296
Generation  7000 :  296
Generation  7200 :  296
Generation  7400 :  296
Generation  7600 :  296
Generation  7800 :  296
Generation  8000 :  296
Generation  8200 : 

In [115]:
result

array([10,  2, 17, 12,  0,  9, 40, 22, 25, 20, 42,  9, 18, 23, 25, 44,  3,
        8, 20, 30,  8, 23,  4, 13, 31, 39, 23, 24,  5, 21, 36,  4, 27, 39,
       14, 41, 14, 38, 19, 24, 19, 33, 19, 29, 11, 19, 27, 44, 43, 27, 34,
       37,  1, 15,  3, 29, 34, 39, 28, 32, 43, 12, 38, 11, 22,  0, 31,  7,
       32, 19, 15, 22,  1, 40, 12, 21, 35, 15, 20, 38, 26, 36, 43, 36, 36,
       19, 31, 32,  3,  2, 22,  2,  4, 14, 24,  2, 26, 24, 14,  7, 11, 37,
       26, 23, 12, 42,  0, 33, 18,  1, 26, 24,  4, 34, 10, 30,  0, 26, 13,
       29, 25, 40, 32, 40,  4,  5, 43, 40, 23, 23, 15, 33, 17, 13, 30, 35,
       22, 26, 16, 42, 22, 20, 28,  1,  1,  8, 32, 34, 37, 31,  4, 10, 15,
       23, 29,  4, 26, 21,  8,  1, 26, 44, 14,  3, 33, 31, 39, 26, 21, 29,
        3, 26, 11, 44,  1, 18, 27, 13, 30, 18, 19, 16,  9, 26,  6,  1, 36,
       38, 44,  9, 11, 12, 20, 41, 26,  3,  8, 37,  5,  6,  7, 42, 43,  8,
       36, 32, 30, 29, 32, 28, 24,  2, 42, 16,  0,  2, 44, 10, 13, 25, 38,
       12, 17, 39, 35,  5

In [116]:
servers

[S[0]: size=2, capacity=40, row=0, slot=0 | P[-1],
 S[1]: size=2, capacity=40, row=0, slot=2 | P[-1],
 S[2]: size=3, capacity=60, row=0, slot=4 | P[-1],
 S[3]: size=2, capacity=40, row=0, slot=7 | P[-1],
 S[4]: size=4, capacity=80, row=0, slot=11 | P[-1],
 S[5]: size=1, capacity=20, row=0, slot=9 | P[-1],
 S[6]: size=5, capacity=100, row=0, slot=15 | P[-1],
 S[7]: size=3, capacity=60, row=0, slot=20 | P[-1],
 S[8]: size=2, capacity=40, row=0, slot=23 | P[-1],
 S[9]: size=3, capacity=60, row=0, slot=25 | P[-1],
 S[10]: size=5, capacity=100, row=0, slot=28 | P[-1],
 S[11]: size=2, capacity=40, row=0, slot=33 | P[-1],
 S[12]: size=3, capacity=60, row=0, slot=35 | P[-1],
 S[13]: size=1, capacity=20, row=0, slot=38 | P[-1],
 S[14]: size=1, capacity=20, row=0, slot=39 | P[-1],
 S[15]: size=3, capacity=60, row=0, slot=43 | P[-1],
 S[16]: size=3, capacity=60, row=0, slot=48 | P[-1],
 S[17]: size=4, capacity=80, row=0, slot=52 | P[-1],
 S[18]: size=4, capacity=80, row=0, slot=56 | P[-1],
 S[19]

In [ ]:
def save_result(servers, pools)